# Gráficos
del rendimiento del sistema. Estos son útiles para comparar tiempos de ejecución entre programas de distinto lenguaje y grado de paralelización

In [3]:
#Cargo librerías
import numpy as np
import matplotlib.pyplot as plt

#Agrego interactividad a los gráficos
%matplotlib ipympl

In [5]:
#Cargo datos.
name_files = ["py_cpu_t_computo.txt", "cpp_cpu_t_computo.txt", "cpp_gpu_v1_t_computo.txt", "cpp_gpu_v2_t_computo.txt"]
# "py_gpu_t_computo.txt"
i = 0
datos = np.load("resultados/" + name_files[i])
# [py_cpu, cpp_cpu, cpp_gpu_v1, cpp_gpu_v2] = [np.loadtxt("resultados/" + name_files[i]) for i in range(len(name_files))]

#Cargo datos


ValueError: Cannot load file containing pickled data when allow_pickle=False

In [ ]:
#Calculo para cada N un promedio y dsv de los datos sin considerar el primer dato de "precalentamiento"

In [ ]:
#Grafico